In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import os
import sys

import `pytrades` and `ancillary` modules

In [ ]:
pytrades_dir = os.path.abspath("../")
sys.path.append(pytrades_dir)

import pytrades
import ancillary as anc
import constants as cst

init `pytrades` with 2p-system

In [ ]:
# exosystem_folder = os.path.abspath("../../trades_example/base_2p/")
exosystem_folder = os.path.abspath("../../trades_example/Kepler-9_example/")
sim = pytrades.TRADES(
    exosystem_folder,
    sub_folder="",
    nthreads=1,
    seed=42,
    m_type="e"
)

In [ ]:
for k, v in vars(sim).items():
    print(k, v)

In [ ]:
sim.init_trades()

define the `delta_sigma`

In [5]:
delta_sigma = 1.0e-3
np.random.seed(666)

function convert to physical parameters some of them

In [6]:
def convert(pars, sim):
    pnew = pars.copy()
    for i, n in enumerate(sim.fitting_names):
        p = pars[i]
        if "Ms" in n:
            pnew[i] = p * sim.MR_star[0,0]*cst.Msear
        if "secosw" in n:
            ps = pars[i+1]
            pnew[i] = p*p + ps*ps
            pnew[i+1] = (np.arctan2(ps, p)*cst.rad2deg)%360.0
        if "l2j" in n:
             pnew[i] = 2.0**p
    return pnew


test original function to compute physical delta

In [ ]:
fit_v1 = convert(sim.fitting_parameters, sim)
delta_v1 = anc.compute_proper_sigma(sim.nfit, delta_sigma, sim.fitting_names)
# par_v1 = delta_v1
par_v1 = np.random.normal(loc=sim.fitting_parameters, scale=delta_v1)
new_v1 = convert(par_v1, sim)
for n, p, pmin, pmax, f, d, pn in zip(
        sim.fitting_names, sim.fitting_parameters, sim.fitting_minmax[:,0], sim.fitting_minmax[:,1],
        fit_v1,
        par_v1, new_v1
    ):
    print(
        "{:12s} = {:16.8f} [{:16.8f} , {:16.8f}] ({:16.8f}) --> {:16.8f} ==> {:16.8f}".format(
        n, p, pmin, pmax, f, d, pn
    ))

test a version based on the boundaries

In [ ]:
# fit_v1 = convert(sim.fitting_parameters, sim)
dp = sim.fitting_minmax[:,1] - sim.fitting_minmax[:,0]
delta_v2 = dp * delta_sigma
# par_v2 = delta_v2
par_v2 = np.random.normal(loc=sim.fitting_parameters, scale=delta_v2)
new_v2 = convert(par_v2, sim)
for n, p, pmin, pmax, f, d, pn in zip(
        sim.fitting_names, sim.fitting_parameters, sim.fitting_minmax[:,0], sim.fitting_minmax[:,1],
        fit_v1,
        par_v2, new_v2
    ):
    
    
    print(
        "{:12s} = {:16.8f} [{:16.8f} , {:16.8f}] ({:16.8f}) --> {:16.8f} ==> {:16.8f}".format(
        n, p, pmin, pmax, f, d, pn
    ))

updated version of `compute_proper_sigma` to test

In [ ]:
def compute_proper_sigma(parameter_names, fitting_minmax, delta_sigma=0.001):


    delta_par = fitting_minmax[:,1] - fitting_minmax[:,0] 

    delta_sigma_out = np.zeros(len(parameter_names))*delta_sigma
    
    for ifit, name in enumerate(parameter_names):
        if "Ms" in name:
            Dx = delta_par[ifit]*cst.Msear
            if Dx < 0.5:
                dx = 0.01
            elif 0.5 <= Dx < 50.0:
                dx = 1.0
            else:
                dx = 10.0    
            delta_sigma_out[ifit] = dx * cst.Mears

        if "P" in name:
            Dx = delta_par[ifit]
            if Dx < 2.0:
                dx = 0.005
            elif 2.0 <= Dx < 10.0:
                dx = 0.25
            else:
                dx = 1.0
            delta_sigma_out[ifit] = dx

        # sqrt(e)cos/sinw is fine with delta_sigma      
        
        if "mA" in name or "lambda" in name or "lN" in name:
            delta_sigma_out[ifit] = 1.0

        if "i" in name:
            delta_sigma_out[ifit] = 0.1

        if "l2j" in name:
            delta_sigma_out[ifit] = np.abs(np.log2(0.5))

        if "gamma" in name:
            delta_sigma_out[ifit] = 10.0

    return delta_sigma_out

In [ ]:
delta_v3 = compute_proper_sigma(sim.fitting_names, sim.fitting_minmax, delta_sigma=delta_sigma)
# par_v3 = delta_v2
par_v3 = np.random.normal(loc=sim.fitting_parameters, scale=delta_v3)
new_v3 = convert(par_v3, sim)
for n, p, pmin, pmax, f, d, pn in zip(
        sim.fitting_names, sim.fitting_parameters, sim.fitting_minmax[:,0], sim.fitting_minmax[:,1],
        fit_v1,
        par_v3, new_v3
    ):
    
    
    print(
        "{:12s} = {:16.8f} [{:16.8f} , {:16.8f}] ({:16.8f}) --> {:16.8f} ==> {:16.8f}".format(
        n, p, pmin, pmax, f, d, pn
    ))

In [ ]:
for n, p0, p1, p2, p3 in zip(sim.fitting_names, fit_v1, new_v1, new_v2, new_v3):

    print(
        "{:12s} : {:16.8f} {:16.8f} {:16.8f} {:16.8f}".format(
        n, p0, p1, p2, p3
    ))